In [5]:
try:
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    import pprint
    import pandas as pd
    import configparser
    print("imported successfully")
except ImportError as e: # If failed to import, return notice
    print("Import faliure:", e)
    
config = configparser.ConfigParser()
config.read('config.ini')

username = config['credentials']['username']
password = config['credentials']['password']

imported successfully


In [13]:
# Initialize splinter/browser
browser = Browser('chrome')

# Visit hymnal
url = 'https://hymnal.pcusastore.com/Hymn/HymnSearch'
browser.visit(url)

browser.fill('UserName', (username))
browser.fill('Password', (password))
browser.find_by_css('.linkbutton-login').first.click() #click the login button

browser.find_by_css('ul.nav a[href="/Hymn/HymnSearch"]').first.click()
# Find the 'Hymn #' link by XPath and click it
hymn_sort_link_xpath = "//a[contains(@class, 'k-link') and contains(text(), 'Hymn #')]"
browser.find_by_xpath(hymn_sort_link_xpath).first.click()

# Click on the first hymn
browser.find_by_css('a[href="/Hymn/Index/1"]').click()

# After logging in, parse the first hymn's HTML with Beautiful Soup
first_hymn_html = browser.html
html_soup = soup(first_hymn_html, 'html.parser')

main_content = html_soup.find('div', class_='mainContent')

# Extract the text nodes (ignoring any nested tags like <h2>, <h3> etc.)
text_lines = [text for text in main_content.stripped_strings]

In [16]:
# Initialize an empty list to store hymn data
hymns_data = []

# for hymn_number in range(2, 4):
for hymn_number in range(2, 854):
    url = f"https://hymnal.pcusastore.com/Hymn/Index/{hymn_number}"
    browser.visit(url)

    # Scrape the hymn title text
    html = browser.html
    html_soup = soup(html, 'html.parser')
    
    hymn_title_element = html_soup.find('h2', class_='hymnTitle')
    hymn_title = hymn_title_element.get_text(strip=True) if hymn_title_element else 'Title not found'

    main_content = html_soup.find('div', class_='mainContent')
    text_lines = ' '.join([text for text in main_content.stripped_strings])
    
    # Store the hymn number and title in a dictionary, then add it to the list
    hymns_data.append({'HymnNumber': hymn_number, 'Title': hymn_title, 'Lyrics': text_lines})
    
df_hymns = pd.DataFrame(hymns_data)

In [18]:
df_hymns.head()

,HymnNumber,Title,Lyrics
0,2,"Come, Thou Almighty King","Previous Hymn Next Hymn Favorite Create Come, ..."
1,3,Womb of Life and Source of Being,Previous Hymn Next Hymn Favorite Create Womb o...
2,4,"Holy God, We Praise Your Name",Previous Hymn Next Hymn Favorite Create Holy G...
3,5,God the Sculptor of the Mountains,Previous Hymn Next Hymn Favorite Create God th...
4,6,I Bind unto Myself Today,Previous Hymn Next Hymn Favorite Create I Bind...


In [26]:
df_hymns.to_csv('/Users/coblehartman/Documents/GitHub/hymnal-web-scraping/unclean_hymns.csv', index=False)

In [35]:
# Load unclean hymn data into a new DataFrame
clean_df_hymns = pd.read_csv('/Users/coblehartman/Documents/GitHub/hymnal-web-scraping/unclean_hymns.csv')

# Define a function to extract text between "Lyrics" and "Informational Notes"
def extract_lyrics(text):
    # Find the start of the lyrics
    start_idx = text.find('Lyrics')
    if start_idx == -1:
        # If 'Lyrics' is not found, return an empty string or the whole text as preferred
        return ''
    start_idx += len('Lyrics')  # Adjust start index to skip the word 'Lyrics' itself
    
    # Find the end of the lyrics
    end_idx = text.find('Informational Notes', start_idx)
    if end_idx == -1:
        # If 'Informational Notes' is not found, return the text from 'Lyrics' to the end
        return text[start_idx:].strip()
    
    return text[start_idx:end_idx].strip()

clean_df_hymns['CleanedText'] = clean_df_hymns['Lyrics'].apply(extract_lyrics)
clean_df_hymns = clean_df_hymns.drop(columns=['Lyrics'])

In [36]:
clean_df_hymns.to_csv('/Users/coblehartman/Documents/GitHub/hymnal-web-scraping/cleaned_hymns.csv', index=False)

In [31]:
# Close the brower's connection
browser.quit()

## Let's import them into PowerPoint now...

In [40]:
try:
    from pptx import Presentation
    from pptx.util import Inches
    print('Import sucessful')
except ImportError as e:
    print('Import Failed', e)

Import sucessful
